# Setup / Imports

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

# Split the ebook into proper chapters

In [4]:
from ebooklib import epub

file1 = "../data/frankenstein.epub"
file2 = "../data/grapes-of-wrath.epub"
file3 = "../data/outlaw-of-torn.epub"
book1 = epub.read_epub(file1, options={'ignore_ncx': True})
book2 = epub.read_epub(file2, options={'ignore_ncx': True})
book3 = epub.read_epub(file3, options={'ignore_ncx': True})

/Users/jimmyyao/miniconda3/envs/novel/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [5]:
from ebook_tree_parser.toctree import TocTree

In [6]:
tree1 = TocTree(book1)
tree2 = TocTree(book2)
tree3 = TocTree(book3)

In [7]:
prev_chapter_content = """The Stonewalls, a British battalion, have been marching towards the front lines, preparing for a major offensive that they refer to as "the push." Rumors abound among the soldiers about what exactly their role will be, but no one is certain of the details. The sound of artillery fire grows louder with each passing day, signaling that the time for action is drawing near.
As they stop to bivouac in an orchard for the night, the battalion is interrupted by enemy artillery shells falling dangerously close to their camp. An artillery subaltern from a nearby British battery warns them that their position is in the direct line of fire from a German battery. He suggests they move to a nearby farm, where they can find some shelter in the damaged barns.
The battalion quickly relocates, and the soldiers settle into the barns, which are battered by previous shellfire but still offer more protection than the open orchard. As they wait for their evening meal, a group of soldiers—including Larry Arundel, Billy Simson, Ben Sneath, and Jefferson Lee—wander outside to watch the artillery battery in action. The guns are wire-cutting, clearing the way for the infantry's advance by destroying the enemy’s barbed wire defenses.
The group observes the operation with a mix of curiosity and awe. Each soldier reacts differently to the spectacle, with Larry explaining the technical aspects of the artillery's work, Billy marveling at the noise and flashes of light, and Pug cracking jokes. Kentuck, meanwhile, reflects quietly, embodying the calm resolve that led him to join the British forces despite his American roots.
As night falls, the guns continue to fire, preparing the battlefield for the "push" that the soldiers know is imminent."""

In [13]:
from novelinsights.prompts.wikigen.fiction import template,excerpt_template

from novelinsights.utils import GET_ESTIMATOR,simple_token_estimator
est=GET_ESTIMATOR(source='openai')

cur_template = excerpt_template(tree2[5])
print(cur_template)

# Task
Your task is to produce a comprehensive and detailed summary of the current chapter, as well as extract and organize all significant information into predefined categories: Characters, Chronology, Locations, Organizations, Things, and Concepts. The output should be thorough and suitable for creating Wikipedia-style entries, ensuring that all essential details from the chapter are captured accurately.

# Instructions
## 1. Information Extraction:
  For each category below, extract relevant details from the chapter. Present the information in a clear and organized manner, using bullet points or numbered lists where appropriate. If the chapter does not contain information for a specific category, you may omit it. And if the content of the chapter is not related to the plot (front matter, acknowledgments, table of contents, etc.), you can skip the extraction for that chapter. If you skip the extraction, add <SKIPPED-EXTRACTION> at the beginning of the section.

   - Entities:
     -

In [30]:
from novelinsights.prompts.schemas import SchemaManager
import json

sm = SchemaManager()

total = 0
for schema in sm:
    print(schema)
    tok = est(json.dumps(schema))
    total += tok
    print(tok)
    # print(simple_token_estimator(json.dumps(schema)))
print(total)

{'type': 'object', 'properties': {'novelInfo': {'type': 'object', 'description': 'Basic information about the novel. Update this section with every new excerpt if necessary', 'properties': {'title': {'type': 'string', 'description': 'The full title of the novel'}, 'author': {'type': 'string', 'description': 'The name of the author(s)'}, 'genre': {'type': 'array', 'items': {'type': 'string'}, 'description': 'List of genres the novel belongs to'}}, 'required': ['title', 'author']}, 'summary': {'type': 'string', 'description': 'A comprehensive summary of the entire story, including key events and developments up to the current excerpt'}, 'characters': {'type': 'array', 'description': 'List of characters in the novel and their key details. Update with new characters every next excerpt', 'items': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': "The character's full name"}, 'description': {'type': 'string', 'description': 'Current physical description and key trai

In [36]:
est(s)

789

# Import openai and get the API key

In [135]:
# api key from .env file
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# from openai import OpenAI
# client = OpenAI(
#   organization='org-ZmPsjhKQZsIYHP2tf06Qw0tb',
#   project='proj_7IW3lRCW3PRtkrALT3OQ7okH',
#   api_key=os.getenv("OPENAI_API_KEY")
# )
# test a call to the API 
# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "user", "content": "write a haiku about ai"}
#     ]
# )
# print(completion.choices[0].message.content)

import anthropic
client = anthropic.Anthropic(
  api_key=os.getenv("ANTHROPIC_API_KEY"),
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)

[TextBlock(text='Briny depths, a mystery old,\nTears of Earth, stories untold.\nRivers carry minerals to sea,\nEons of salt, set eternally free.', type='text')]


In [128]:
import json
def parse_json(response:str, json_index:int=0) -> dict:
    # json is between ``` and ``` in the response, possibly with a ```json and ``` as well
    # the json_index json surrounded by ``` is the one we want
    # repeatedly find ``` until we find the json_index-th one
    # use response.find
    """ Find the json_index-th json in the response

    Args:
        response (str): The string response from the API
        json_index (int, optional): index of the json to find. Defaults to 0.

    Returns:
        dict: The json object. Could error if the json is not valid
    
    Throws:
        ValueError: If the json is not
    """
    start = -1
    count = 0
    while count < json_index+1:
        start = response.find("```", start+1)
        if start == -1:
            break
        count += 1
    # find the end of the json
    end = response.find("```", start+1)
 
    if start == -1 or end == -1:
        return None
    
    # handle the case where there is a ```json\n and ``` in the response
    if response[start+3:start+7] == "json":
        start += 7

    return json.loads(response[start:end])
    
parse_json('rw```json\n{"gi":"ffi"}```')

{'gi': 'ffi'}

# Prepare folders to store outputs

In [127]:
# make directory for the output, if it exists, create a new one with incremented number
import os

output_dir = "../output/frankenstein"
if os.path.exists(output_dir):
    i = 1
    while os.path.exists(output_dir + "_" + str(i)):
        i += 1
    output_dir += "_" + str(i)
os.makedirs(output_dir)

# make folders "inprompt", "json", and "response" folders in the output directory
os.makedirs(os.path.join(output_dir, "inprompt"))
os.makedirs(os.path.join(output_dir, "json"))
os.makedirs(os.path.join(output_dir, "response"))

In [57]:
# message = client.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=1000,
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": template(tree[0].content)
#                 }
#             ]
#         }
#     ]
# )
# print(message.content[0].text)

Here's the JSON created based on the provided excerpt and schema:

```json
{
  "novelInfo": {
    "title": "Frankenstein; Or, The Modern Prometheus",
    "author": "Mary Wollstonecraft Shelley",
    "genre": ["Gothic", "Science Fiction", "Horror"]
  },
  "summary": "Frankenstein; Or, The Modern Prometheus is a novel that explores themes of scientific ambition, the consequences of creating life, and the responsibilities of a creator to their creation. The story likely follows the journey of a scientist who creates a living being, leading to unforeseen and potentially disastrous consequences.",
  "characters": [
    {
      "name": "Victor Frankenstein",
      "description": "Presumed to be the main character, a scientist who likely creates artificial life.",
      "role": "protagonist"
    },
    {
      "name": "The Creature",
      "description": "Presumed to be the artificial being created by Victor Frankenstein.",
      "role": "antagonist"
    }
  ],
  "plotPoints": [],
  "worldBui

In [46]:
# completion = client.chat.completions.create(
#     model="chatgpt-4o-latest",
#     messages=[
#         {"role": "user", "content": template(tree[0].content)}
#     ]
# )
# print(completion.choices[0].message.content)

In [129]:
from novelinsights.prompts.wikigen.fiction import template

prev_json = None
for i, excerpt in enumerate(tree):
    print(f"[{i}]\t{excerpt.title}")
    
    cur_template = template(excerpt, prev_json=prev_json)
    
    with open(os.path.join(output_dir, "inprompt", f"[{i}] {excerpt.title}.txt"), "w") as f:
        f.write(cur_template)
    
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=8192,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": cur_template
                        }
                    ]
                }
            ]
        )
        
        with open(os.path.join(output_dir, "response", f"[{i}] {excerpt.title}.txt"), "w") as f:
            f.write(message.content[0].text)
        
        try:
            prev_json = parse_json(message.content[0].text)
        except:
            pass
        
        with open(os.path.join(output_dir, "json", f"[{i}] {excerpt.title}.json"), "w") as f:
            f.write(json.dumps(prev_json, indent=4))
            

[0]	Frankenstein;
[1]	or, the Modern Prometheus
[2]	CONTENTS
[3]	Letter 1
[4]	Letter 2
[5]	Letter 3
[6]	Letter 4
[7]	Chapter 1
[8]	Chapter 2
[9]	Chapter 3
[10]	Chapter 4
[11]	Chapter 5
[12]	Chapter 6


KeyboardInterrupt: 